In [1]:
import copy
import numpy as np
from skimage.segmentation import watershed,clear_border
import scipy.misc
from skimage.io import imread
from matplotlib import pyplot as plt
import scipy.ndimage as ndi
import pickle
import os
from os import listdir
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects,local_maxima,h_maxima,opening
from skimage import measure
from skimage.measure import regionprops,label
from skimage.color import label2rgb
from PIL import Image, ImageDraw, ImageFont
from math import pi,sqrt
# import cv2
import glob
import pandas as pd
import seaborn as sns

from sklearn import decomposition,cluster,manifold
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler,RobustScaler,Normalizer
from cell_class import single_cell,fluor_single_cell

In [4]:
main_path='2021-03-25_a549_1ng_tgf_78h/'
cells_path='fluor_cells/'
posi_start=1
posi_end=20

In [12]:
all_data=np.array([])
for posi in range(posi_start,posi_end+1): 
    with open (cells_path+'fluor_cells_'+str(posi), 'rb') as fp:
        cells = pickle.load(fp)
    data=np.array([single_cell.vimentin_feature_values[4] for single_cell in cells if hasattr(single_cell,'vimentin_feature_values')])

    
    if all_data.size==0:
        all_data=data
    else:
        all_data=np.vstack((all_data,data))

scaler = StandardScaler()

X=scaler.fit_transform(all_data)
print(X.shape)

pca = decomposition.PCA(n_components =0.98,svd_solver= 'full')
Y = pca.fit_transform(X)
print(pca.components_,pca.explained_variance_ratio_)

(495952, 13)
[[ 0.36273021  0.28215444 -0.32120511 -0.26156484  0.00881163  0.02813715
  -0.27330279 -0.36836584 -0.28277291  0.252443   -0.28252768  0.33094456
  -0.27006564]
 [ 0.05349904 -0.35249758  0.27317291 -0.09870011  0.51250534 -0.50753188
  -0.07658333 -0.06007493  0.02048958  0.36452016 -0.30866619 -0.06361377
   0.1617947 ]
 [ 0.18389119  0.13395517 -0.04703356  0.57916422  0.16932493  0.09043871
   0.561272   -0.08208859 -0.32301285  0.20969107 -0.14785972 -0.23680404
  -0.1604229 ]
 [-0.04624818  0.06698877 -0.27234565  0.02318244  0.33513328 -0.09737025
   0.01890184  0.2709133   0.52857774  0.010461    0.02730513 -0.01192671
  -0.66537642]
 [-0.16605103  0.10048101  0.08712766  0.11984123  0.0069179   0.46804027
   0.11193045  0.0528059   0.4414728   0.35426556 -0.38622352  0.39520263
   0.28318719]
 [-0.08020712  0.21739234 -0.13979993 -0.23356044  0.47986578  0.46267054
  -0.24204315  0.11457763 -0.13325002  0.18725426  0.18018876 -0.47907496
   0.20338203]
 [ 0.4297

In [13]:
with open('norm_haralick_pca', 'wb') as fp:
    pickle.dump(pca, fp)

In [14]:
with open('norm_haralick_pca', 'rb') as fp:
    pca=pickle.load(fp)

In [15]:
fluor_feature_name='norm_vimentin_haralick'
for posi in range(posi_start,posi_end+1):   
    with open (cells_path+'fluor_cells_'+str(posi), 'rb') as fp:
        cells = pickle.load(fp)
    for i in range(len(cells)):
        if hasattr(cells[i],'vimentin_feature_values'):
            X=np.expand_dims(cells[i].vimentin_feature_values[4],axis=0)
            X=scaler.transform(X)
            Y=pca.transform(X)[0]
            cells[i].set_fluor_pca_cord(fluor_feature_name,Y)
    with open(cells_path+'fluor_cells_'+str(posi), 'wb') as fp:
        pickle.dump(cells, fp)

In [16]:
# with open (cells_path+'fluor_cells_1', 'rb') as fp:
#     cells = pickle.load(fp)

print(cells[0].norm_vimentin_haralick_pca_cord.shape)

(7,)
